In [ ]:
pip install torch torchvision timm pandas requests

In [2]:
from PIL import Image
import torch
import timm
import requests
import torchvision.transforms as transforms
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

print(torch.__version__)
# # should be 1.8.0


# model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
# model.eval()

# transform = transforms.Compose([
#     transforms.Resize(256, interpolation=3),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
# ])

# img = Image.open(requests.get("https://raw.githubusercontent.com/pytorch/ios-demo-app/master/HelloWorld/HelloWorld/HelloWorld/image.png", stream=True).raw)
# img = transform(img)[None,]
# out = model(img)
# clsidx = torch.argmax(out)
# print(clsidx.item())

2.4.0


In [ ]:
# model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
# model.eval()
# scripted_model = torch.jit.script(model)
# scripted_model.save("fbdeit_scripted.pt")

In [ ]:
# # Use 'x86' for server inference (the old 'fbgemm' is still available but 'x86' is the recommended default) and ``qnnpack`` for mobile inference.
# backend = "x86" # replaced with ``qnnpack`` causing much worse inference speed for quantized model on this notebook
# model.qconfig = torch.quantization.get_default_qconfig(backend)
# torch.backends.quantized.engine = backend

# quantized_model = torch.quantization.quantize_dynamic(model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8)
# scripted_quantized_model = torch.jit.script(quantized_model)
# scripted_quantized_model.save("fbdeit_scripted_quantized.pt")

In [ ]:
# out = scripted_quantized_model(img)
# clsidx = torch.argmax(out)
# print(clsidx.item())


In [ ]:
# from torch.utils.mobile_optimizer import optimize_for_mobile
# optimized_scripted_quantized_model = optimize_for_mobile(scripted_quantized_model)
# optimized_scripted_quantized_model.save("fbdeit_optimized_scripted_quantized.pt")

In [ ]:
# out = optimized_scripted_quantized_model(img)
# clsidx = torch.argmax(out)
# print(clsidx.item())


In [ ]:
# optimized_scripted_quantized_model._save_for_lite_interpreter("fbdeit_optimized_scripted_quantized_lite.ptl")
# ptl = torch.jit.load("fbdeit_optimized_scripted_quantized_lite.ptl")

In [ ]:
# with torch.autograd.profiler.profile(use_cuda=False) as prof1:
#     out = model(img)
# with torch.autograd.profiler.profile(use_cuda=False) as prof2:
#     out = scripted_model(img)
# with torch.autograd.profiler.profile(use_cuda=False) as prof3:
#     out = scripted_quantized_model(img)
# with torch.autograd.profiler.profile(use_cuda=False) as prof4:
#     out = optimized_scripted_quantized_model(img)
# with torch.autograd.profiler.profile(use_cuda=False) as prof5:
#     out = ptl(img)

# print("original model: {:.2f}ms".format(prof1.self_cpu_time_total/1000))
# print("scripted model: {:.2f}ms".format(prof2.self_cpu_time_total/1000))
# print("scripted & quantized model: {:.2f}ms".format(prof3.self_cpu_time_total/1000))
# print("scripted & quantized & optimized model: {:.2f}ms".format(prof4.self_cpu_time_total/1000))
# print("lite model: {:.2f}ms".format(prof5.self_cpu_time_total/1000))

# Fine Tuning First...

In [3]:
import os
import zipfile
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
random_seed=42
torch.manual_seed(random_seed)
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import accuracy_score
np.random.seed(random_seed)

import torch
import torch.optim as optim
import torch.nn.functional as F

In [4]:
#@title "Data Prep"
root_path='/kaggle/input/kather2016/Kather_texture_2016_image_tiles_5000'
#------------------------------------------------------------------------------------------------------------------------------------------------------------------
tumors,stroma,complex,lympho,debris,mucosa,adipose,empty=[],[],[],[],[],[],[],[]
for index in range(8):
  if index==0:
    tumors=tumors+(os.listdir(root_path+'/01_TUMOR'))
    tumors=[root_path+'/01_TUMOR/'+item for item in tumors]
  elif index==1:
    stroma+=(os.listdir(root_path+'/02_STROMA'))
    stroma=[root_path+'/02_STROMA/'+item for item in stroma]
  elif index==2:
    complex+=(os.listdir(root_path+'/03_COMPLEX'))
    complex=[root_path+'/03_COMPLEX/'+item for item in complex]
  elif index==3:
    lympho+=(os.listdir(root_path+'/04_LYMPHO'))
    lympho=[root_path+'/04_LYMPHO/'+item for item in lympho]
  elif index==4:
    debris+=(os.listdir(root_path+'/05_DEBRIS'))
    debris=[root_path+'/05_DEBRIS/'+item for item in debris]
  elif index==5:
    mucosa+=(os.listdir(root_path+'/06_MUCOSA'))
    mucosa=[root_path+'/06_MUCOSA/'+item for item in mucosa]
  elif index==6:
    adipose+=(os.listdir(root_path+'/07_ADIPOSE'))
    adipose=[root_path+'/07_ADIPOSE/'+item for item in adipose]
  else:
    empty+=(os.listdir(root_path+'/08_EMPTY'))
    empty=[root_path+'/08_EMPTY/'+item for item in empty]
      
print('Tumors {}\nStroma {}\nComplex {}\nLympho {}\nDebris {}\n Mucosa {}\n Adipose {}\n Empty {}'.format(len(tumors),len(stroma),len(complex),len(lympho),len(debris),len(mucosa),len(adipose),len(empty)))

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#Creating dataframe
all_files=tumors+stroma+complex+lympho+debris+mucosa+adipose+empty
labels=[]
for _ in range(len(tumors)):
  # labels.append(1)
  labels.append('tumor')
for _ in range(len(stroma)):
  # labels.append(2)
  labels.append('stroma')
for _ in range(len(complex)):
  # labels.append(3)
  labels.append('complex')
for _ in range(len(lympho)):
  # labels.append(4)
  labels.append('lympho')
for _ in range(len(debris)):
  # labels.append(5)
  labels.append('debris')
for _ in range(len(mucosa)):
  # labels.append(6)
  labels.append('mucosa')
for _ in range(len(adipose)):
  # labels.append(7)
  labels.append('adipose')
for _ in range(len(empty)):
  # labels.append(8)
  labels.append('empty')

train_df=pd.DataFrame({
    "Filename":all_files,
    "Labels":labels
})
train_df['Labels'] = train_df['Labels'].astype(str)

#------------------------------------------------------------------------------------------------------------------------------------------------------------------
test_splits=[.9,.8,.75,.7,.65,.6]

for split in test_splits:
    
  train_df_,test_df_=train_test_split(train_df,test_size=split,random_state=random_seed,stratify=train_df['Labels'],shuffle=True)
  test_df_,val_df_=train_test_split(test_df_,test_size=0.2,random_state=random_seed,stratify=test_df_['Labels'],shuffle=True)
  test_df_.reset_index(drop=True,inplace=True)
  train_df_.reset_index(drop=True,inplace=True)
  val_df_.reset_index(drop=True,inplace=np.True_)
    
  train_df_.to_csv('Training_point{}.csv'.format(int(split*100)),index=False)
  test_df_.to_csv('Test_point{}.csv'.format(int(split*100)),index=False)
  val_df_.to_csv('Val_point{}.csv'.format(int(split*100)),index=False)
    

Tumors 625
Stroma 625
Complex 625
Lympho 625
Debris 625
 Mucosa 625
 Adipose 625
 Empty 625


In [11]:
#@title "Load train,test,val"
def read_csv(path:str):
  df=pd.read_csv(path)
  return df

train_df=read_csv('Training_point90.csv')
test_df=read_csv('Test_point90.csv')
val_df=read_csv('Val_point90.csv')

labels=sorted(list(train_df['Labels'].unique()))

label2idx={}
idx2label={}

for idx,label in enumerate(labels):
  label2idx[label]=idx
  idx2label[idx]=label

train_df['Labels']=train_df['Labels'].map(label2idx)
test_df['Labels']=test_df['Labels'].map(label2idx)
val_df['Labels']=val_df['Labels'].map(label2idx)

print(f''' 
Training data: {train_df.shape[0]}\n
Test data : {test_df.shape[0]}\n
Validation data : {val_df.shape[0]}
''')

 
Training data: 500

Test data : 3600

Validation data : 900



In [6]:
teacher = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
student = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)

Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main
Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main


In [7]:
transform = transforms.Compose([
    transforms.Resize(256, interpolation=3),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
])


In [12]:
class TissueData(Dataset):
  @staticmethod
  def read_img(path:str):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(224,224))
    img=img.astype(np.float32)
    img=np.transpose(img,(2,0,1))
    img=img/255.
    return img
  def __init__(self,df:pd.DataFrame):
    self.df=df
  def __len__(self):
    return len(self.df)
  def __getitem__(self,idx):
    img_path=self.df.iloc[idx,0]
    img=self.read_img(img_path)
    label=int(self.df['Labels'][idx])
    return img,label

#@title "Custom Dataset Unlabeled"
class UnlabeledTissueData(Dataset):
  @staticmethod
  def read_img(path:str):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(224,224))
    img=img.astype(np.float32)
    img=np.transpose(img,(2,0,1))
    img=img/255.
    return img
      
  def __init__(self,df:pd.DataFrame):
    self.df=df
  def __len__(self):
    return len(self.df)
  def __getitem__(self,idx):
    img_path=self.df.iloc[idx,0]
    img=self.read_img(img_path)
    return img

In [13]:
params={
    'batch_size':16,
    'shuffle':True,
    'num_workers':4,
    'pin_memory':True,
    'epochs':20,
    'lr':0.001,
    'device':'cuda' if torch.cuda.is_available() else 'cpu'
}
labeled_ds=TissueData(train_df)
labeled_loader=DataLoader(labeled_ds,batch_size=params['batch_size'],shuffle=True)
val_ds=TissueData(val_df)

In [14]:
#Remove labels from test_df
unlabelled_df=test_df.drop('Labels',axis=1)
unlabelled_df=unlabelled_df.reset_index(drop=True)
unlabeled_loader=DataLoader(UnlabeledTissueData(unlabelled_df),batch_size=params['batch_size'],shuffle=False)
val_loader=DataLoader(val_ds,batch_size=params['batch_size'],shuffle=False)

In [15]:
num_epochs=params['epochs']
# Optimizers
t_optimizer = optim.SGD(teacher.parameters(), lr=0.001,momentum=0.9)
s_optimizer = optim.SGD(student.parameters(), lr=0.001,momentum=0.9)

# Lists to store metrics
train_losses = []
train_accuracies = []

val_losses = []
val_accuracies = []

In [16]:
def compute_cosine_similarity(grad1, grad2):
    """
    Computes the cosine similarity between two gradient dictionaries.
    
    Args:
        grad1 (dict): Gradient dictionary 1.
        grad2 (dict): Gradient dictionary 2.
    
    Returns:
        float: Cosine similarity score.
    """
    cosine_similarity = 0.0
    num_valid_keys = 0  # To track valid gradient pairs

    for key in grad1.keys():
        g1, g2 = grad1[key], grad2[key]
        
        # Ensure both gradients are valid tensors
        if g1 is not None and g2 is not None and g1.numel() > 0 and g2.numel() > 0:
            dot_product = torch.sum(g1 * g2)
            norm_g1 = torch.norm(g1)
            norm_g2 = torch.norm(g2)

            if norm_g1 > 0 and norm_g2 > 0:  # Avoid division by zero
                cosine_similarity += (dot_product / (norm_g1 * norm_g2)).item()
                num_valid_keys += 1

    # Average cosine similarity over all valid keys
    return cosine_similarity / num_valid_keys if num_valid_keys > 0 else 0.0

In [17]:
from itertools import cycle
teacher.to(params['device'])
student.to(params['device'])
for epoch in range(num_epochs):
    # Initialize metrics
    epoch_train_loss, epoch_train_correct, total_train_samples = 0.0, 0, 0
    epoch_val_loss, epoch_val_correct, total_val_samples = 0.0, 0, 0

    teacher.train()
    student.train()

    mini_batch_counter = 0
    n_batches = 10

    # Iterate over labeled and unlabeled data
    for (labeled_batch, unlabeled_batch) in zip(labeled_loader, cycle(unlabeled_loader)):
        mini_batch_counter += 1

        # Load labeled data
        l_x, l_y = labeled_batch
        l_x, l_y = l_x.to(params['device']), l_y.to(params['device'])

        # Load unlabeled data
        u_x = unlabeled_batch
        u_x = u_x.to(params['device'])

        # Zero gradients for both optimizers
        t_optimizer.zero_grad()
        s_optimizer.zero_grad()

        # ----------------------------------
        # Pseudo-labeling by Teacher
        # ----------------------------------
        y_u_pred_logits = teacher(u_x)
        y_u_pred = torch.argmax(y_u_pred_logits.detach(), dim=1)

        # ----------------------------------
        # Student trains on pseudo-labeled data
        # ----------------------------------
        s_u_logits = student(u_x)
        s_u_loss = F.cross_entropy(s_u_logits, y_u_pred)
        s_u_loss.backward()

        # Store gradients for unlabeled data
        student_unlabeled_gradients = {
            name: param.grad.clone() if param.grad is not None else None
            for name, param in student.named_parameters()
        }

        s_optimizer.step()

        # ----------------------------------
        # Student trains on labeled data
        # ----------------------------------
        s_optimizer.zero_grad()
        s_l_logits = student(l_x)
        s_l_loss = F.cross_entropy(s_l_logits, l_y)
        s_l_loss.backward()

        # Store gradients for labeled data
        student_labeled_gradients = {
            name: param.grad.clone() if param.grad is not None else None
            for name, param in student.named_parameters()
        }

        s_optimizer.zero_grad()

        # Compute cosine similarity
        h = compute_cosine_similarity(student_labeled_gradients, student_unlabeled_gradients)

        # ----------------------------------
        # Teacher meta-loss (MPL)
        # ----------------------------------
        t_l_logits = teacher(l_x)
        t_l_loss = F.cross_entropy(t_l_logits, l_y)
        t_mpl_loss = h * F.cross_entropy(y_u_pred_logits, y_u_pred)

        (t_l_loss + t_mpl_loss).backward()
        t_optimizer.step()

        # Track training metrics
        batch_train_loss = s_u_loss.item()
        batch_train_accuracy = (torch.argmax(s_l_logits, dim=1) == l_y).float().mean().item()

        epoch_train_loss += batch_train_loss * l_x.size(0)
        epoch_train_correct += (torch.argmax(s_l_logits, dim=1) == l_y).sum().item()
        total_train_samples += l_x.size(0)

        # Print metrics every `n_batches`
        if mini_batch_counter % n_batches == 0:
            avg_batch_train_loss = epoch_train_loss / total_train_samples
            avg_batch_train_accuracy = epoch_train_correct / total_train_samples

            print(f'[Epoch {epoch+1}, Batch {mini_batch_counter}] '
                  f'Batch Loss: {avg_batch_train_loss:.4f}, Batch Accuracy: {avg_batch_train_accuracy:.4f}')

    # ----------------------------------
    # Validation after each epoch
    # ----------------------------------
    teacher.eval()
    student.eval()
    with torch.no_grad():
        for val_batch in val_loader:
            val_x, val_y = val_batch
            val_x, val_y = val_x.to(params['device']), val_y.to(params['device'])

            val_logits = student(val_x)
            val_loss = F.cross_entropy(val_logits, val_y)

            epoch_val_loss += val_loss.item() * val_x.size(0)
            epoch_val_correct += (torch.argmax(val_logits, dim=1) == val_y).sum().item()
            total_val_samples += val_x.size(0)

    # Average losses and accuracies
    avg_train_loss = epoch_train_loss / total_train_samples
    avg_train_accuracy = epoch_train_correct / total_train_samples
    avg_val_loss = epoch_val_loss / total_val_samples
    avg_val_accuracy = epoch_val_correct / total_val_samples

    # Append metrics
    train_losses.append(avg_train_loss)
    train_accuracies.append(avg_train_accuracy)
    val_losses.append(avg_val_loss)
    val_accuracies.append(avg_val_accuracy)

    # Print epoch summary
    print(f'End of Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {avg_train_loss:.4f}, Train Accuracy: {avg_train_accuracy:.4f}, '
          f'Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}')

[Epoch 1, Batch 10] Batch Loss: 1.8198, Batch Accuracy: 0.1187


KeyboardInterrupt: 

In [ ]:
# model.qconfig=torch.quantization.get_default_qconfig('fbgemm')
# model=torch.quantization.prepare_qat(model, inplace=True)

# QAT

In [30]:
%%capture capt
teacher_quantized = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
teacher_quantized.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
torch.quantization.prepare_qat(teacher_quantized, inplace=True)
# teacher = teacher.cpu()
# torch.quantization.convert(teacher, inplace=True)
# print_size_of_model(teacher)

In [32]:
%%capture capt
student_quantized = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
student_quantized.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
torch.quantization.prepare_qat(student_quantized,inplace=True)
# teacher = teacher.cpu()
# torch.quantization.convert(teacher, inplace=True)
# print_size_of_model(teacher)

In [33]:
num_epochs=params['epochs']
# Optimizers
t_optimizer = optim.SGD(teacher_quantized.parameters(), lr=0.001,momentum=0.9)
s_optimizer = optim.SGD(student_quantized.parameters(), lr=0.001,momentum=0.9)

# Lists to store metrics
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

In [34]:
from itertools import cycle
teacher_quantized.to(params['device'])
student_quantized.to(params['device'])
for epoch in range(num_epochs):
    # Initialize metrics
    epoch_train_loss, epoch_train_correct, total_train_samples = 0.0, 0, 0
    epoch_val_loss, epoch_val_correct, total_val_samples = 0.0, 0, 0

    teacher.train()
    student.train()

    mini_batch_counter = 0
    n_batches = 10

    # Iterate over labeled and unlabeled data
    for (labeled_batch, unlabeled_batch) in zip(labeled_loader, cycle(unlabeled_loader)):
        mini_batch_counter += 1

        # Load labeled data
        l_x, l_y = labeled_batch
        l_x, l_y = l_x.to(params['device']), l_y.to(params['device'])

        # Load unlabeled data
        u_x = unlabeled_batch
        u_x = u_x.to(params['device'])

        # Zero gradients for both optimizers
        t_optimizer.zero_grad()
        s_optimizer.zero_grad()

        # ----------------------------------
        # Pseudo-labeling by Teacher
        # ----------------------------------
        y_u_pred_logits = teacher_quantized(u_x)
        y_u_pred = torch.argmax(y_u_pred_logits.detach(), dim=1)

        # ----------------------------------
        # Student trains on pseudo-labeled data
        # ----------------------------------
        s_u_logits = student_quantized(u_x)
        s_u_loss = F.cross_entropy(s_u_logits, y_u_pred)
        s_u_loss.backward()

        # Store gradients for unlabeled data
        student_unlabeled_gradients = {
            name: param.grad.clone() if param.grad is not None else None
            for name, param in student_quantized.named_parameters()
        }

        s_optimizer.step()

        # ----------------------------------
        # Student trains on labeled data
        # ----------------------------------
        s_optimizer.zero_grad()
        s_l_logits = student_quantized(l_x)
        s_l_loss = F.cross_entropy(s_l_logits, l_y)
        s_l_loss.backward()

        # Store gradients for labeled data
        student_labeled_gradients = {
            name: param.grad.clone() if param.grad is not None else None
            for name, param in student_quantized.named_parameters()
        }

        s_optimizer.zero_grad()

        # Compute cosine similarity
        h = compute_cosine_similarity(student_labeled_gradients, student_unlabeled_gradients)

        # ----------------------------------
        # Teacher meta-loss (MPL)
        # ----------------------------------
        t_l_logits = teacher_quantized(l_x)
        t_l_loss = F.cross_entropy(t_l_logits, l_y)
        t_mpl_loss = h * F.cross_entropy(y_u_pred_logits, y_u_pred)

        (t_l_loss + t_mpl_loss).backward()
        t_optimizer.step()

        # Track training metrics
        batch_train_loss = s_u_loss.item()
        batch_train_accuracy = (torch.argmax(s_l_logits, dim=1) == l_y).float().mean().item()

        epoch_train_loss += batch_train_loss * l_x.size(0)
        epoch_train_correct += (torch.argmax(s_l_logits, dim=1) == l_y).sum().item()
        total_train_samples += l_x.size(0)

        # Print metrics every `n_batches`
        if mini_batch_counter % n_batches == 0:
            avg_batch_train_loss = epoch_train_loss / total_train_samples
            avg_batch_train_accuracy = epoch_train_correct / total_train_samples

            print(f'[Epoch {epoch+1}, Batch {mini_batch_counter}] '
                  f'Batch Loss: {avg_batch_train_loss:.4f}, Batch Accuracy: {avg_batch_train_accuracy:.4f}')

    # ----------------------------------
    # Validation after each epoch
    # ----------------------------------
    teacher.eval()
    student.eval()
    with torch.no_grad():
        for val_batch in val_loader:
            val_x, val_y = val_batch
            val_x, val_y = val_x.to(params['device']), val_y.to(params['device'])

            val_logits = student_quantized(val_x)
            val_loss = F.cross_entropy(val_logits, val_y)

            epoch_val_loss += val_loss.item() * val_x.size(0)
            epoch_val_correct += (torch.argmax(val_logits, dim=1) == val_y).sum().item()
            total_val_samples += val_x.size(0)

    # Average losses and accuracies
    avg_train_loss = epoch_train_loss / total_train_samples
    avg_train_accuracy = epoch_train_correct / total_train_samples
    avg_val_loss = epoch_val_loss / total_val_samples
    avg_val_accuracy = epoch_val_correct / total_val_samples

    # Append metrics
    train_losses.append(avg_train_loss)
    train_accuracies.append(avg_train_accuracy)
    val_losses.append(avg_val_loss)
    val_accuracies.append(avg_val_accuracy)

    # Print epoch summary
    print(f'End of Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {avg_train_loss:.4f}, Train Accuracy: {avg_train_accuracy:.4f}, '
          f'Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}')

[Epoch 1, Batch 10] Batch Loss: 2.5368, Batch Accuracy: 0.0875
[Epoch 1, Batch 20] Batch Loss: 3.0939, Batch Accuracy: 0.1437
[Epoch 1, Batch 30] Batch Loss: 2.9490, Batch Accuracy: 0.1958
End of Epoch 1/20, Train Loss: 2.8964, Train Accuracy: 0.2000, Val Loss: 2.6905, Val Accuracy: 0.3967
[Epoch 2, Batch 10] Batch Loss: 1.6184, Batch Accuracy: 0.3875
[Epoch 2, Batch 20] Batch Loss: 1.8269, Batch Accuracy: 0.3375
[Epoch 2, Batch 30] Batch Loss: 1.8716, Batch Accuracy: 0.3500
End of Epoch 2/20, Train Loss: 1.8417, Train Accuracy: 0.3480, Val Loss: 1.4932, Val Accuracy: 0.4389
[Epoch 3, Batch 10] Batch Loss: 1.2907, Batch Accuracy: 0.4375
[Epoch 3, Batch 20] Batch Loss: 1.3497, Batch Accuracy: 0.4562
[Epoch 3, Batch 30] Batch Loss: 1.3294, Batch Accuracy: 0.4604
End of Epoch 3/20, Train Loss: 1.3348, Train Accuracy: 0.4600, Val Loss: 1.2915, Val Accuracy: 0.5167
[Epoch 4, Batch 10] Batch Loss: 0.9420, Batch Accuracy: 0.6500
[Epoch 4, Batch 20] Batch Loss: 1.0395, Batch Accuracy: 0.6062
[

In [37]:
def print_size_of_model(model):
    """ Prints the real size of the model """
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [52]:
teacher_quantized = teacher_quantized.cpu()
torch.quantization.convert(teacher_quantized, inplace=True)
print_size_of_model(teacher_quantized)

Size (MB): 88.862924


In [53]:
torch.save(teacher_quantized.state_dict(),"teacher_quantized_model.pt")

In [57]:
teacher = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
teacher.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
torch.quantization.prepare_qat(teacher, inplace=True)
torch.quantization.convert(teacher,inplace=True)
teacher.load_state_dict(torch.load("teacher_quantized_model.pt"))

Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main
/tmp/ipykernel_23/3932794387.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_di

<All keys matched successfully>

In [60]:
teacher.to('cpu')
teacher.train()
for img,label in val_loader:
    img=img.cpu()
    label=label.cpu()
    out=teacher(img)
    break

NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

Meta: registered at /usr/local/src/pytorch/aten/src/AT

In [51]:
print_size_of_model(teacher)

Size (MB): 347.976818


In [41]:
model=torch.load_state_dict('/kaggle/working/teacher_quantized_model.pt')

/tmp/ipykernel_23/1427919192.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model=torch.load('/kaggle/working/teacher_quantized_model.pt')


In [ ]:
student=student.cpu()
torch.quantization.convert(student,inplace=True)
print_size_of_model(student)

In [ ]:
import torch
from torch.utils.data import DataLoader
criterion=nn.CrossEntropyLoss()
def evaluate_model(model, val_loader, criterion, device='cpu'):
    """
    Evaluate the model on a validation dataset.
    
    Args:
    - model: The PyTorch model to evaluate.
    - val_loader: DataLoader for the validation dataset.
    - criterion: The loss function.
    - device: The device to run the evaluation on ('cpu' or 'cuda').
    
    Returns:
    - avg_loss: Average loss across the validation set.
    - accuracy: Accuracy across the validation set.
    """
    model.eval()  # Set model to evaluation mode
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # No need to compute gradients
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)  # Accumulate loss
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)  # Get class predictions
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    # Compute average loss and accuracy
    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples

    return avg_loss, accuracy

# Example usage
# Assuming val_loader, model, and criterion are defined
# Move the model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

avg_loss, accuracy = evaluate_model(model, val_loader, criterion, device)
print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2%}")


In [ ]:
model

In [ ]:
teacher